In [1]:
import requests
from scrapy.http import TextResponse
import pandas as pd
import datetime

In [2]:
title_ls,link_ls = [],[]
for page in range(1,21+1):
    url = 'http://m.finance.daum.net/m/news/news_list.daum?type=main&page={}'.format(page)
    req = requests.get(url)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    for content in range(1,10+1):
        try : 
            title_ls.append(response.xpath('//*[@id="mArticle"]/div[2]/ul/li[{}]/a/strong/text()'.format(content)).extract()[0])
            link_ls.append(response.xpath('//*[@id="mArticle"]/div[2]/ul/li[{}]/a/@href'.format(content)).extract()[0])
        except : pass

In [3]:
len(title_ls) == len(link_ls)

True

In [4]:
df = pd.DataFrame()
df['title'] = title_ls
df['link'] = link_ls
df['link'] = ['http://m.finance.daum.net' + i for i in df['link']]
df.tail()

,title,link
184,유류세 인하 일주일..휘발유 110원·경유 73원 내려,http://m.finance.daum.net/m/news/news_content....
185,[단독]'라면 마케팅' 비용 메우려 과자 54개 제품 가격 올리는 농심(종합),http://m.finance.daum.net/m/news/news_content....
186,10월 코스닥 개인 거래비중 80%..약 20년만에 최저,http://m.finance.daum.net/m/news/news_content....
187,"[단독]새우깡도 오른다..농심, 과자 20개 브랜드 '가격 최대 9.1% 인상'",http://m.finance.daum.net/m/news/news_content....
188,대의원에게 소주 1박스·쌀 1포대 전달..조합 임원 '당선무효형',http://m.finance.daum.net/m/news/news_content....


In [5]:
content_ls = []

for i in df['link'].values:
    req = requests.get(i)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    content_ls.append(','.join(response.xpath('//*[@id="dmcfContents"]/section/p/text()').extract()).replace(',',''))

In [6]:
len(title_ls) == len(link_ls) == len(content_ls)

True

In [7]:
df['content'] = content_ls
df['content'] = [i.replace("\t",'').replace('\n','') for i in df['content'].values]
df.tail()

,title,link,content
184,유류세 인하 일주일..휘발유 110원·경유 73원 내려,http://m.finance.daum.net/m/news/news_content....,【서울=뉴시스】김지은 기자 = 휘발유와 경유 등에 붙는 세금이 15% 인하된지 일주...
185,[단독]'라면 마케팅' 비용 메우려 과자 54개 제품 가격 올리는 농심(종합),http://m.finance.daum.net/m/news/news_content....,[아시아경제 이선애 기자] 주요 식품 가격이 줄줄이 오르고 있는 가운데 과자 가격도...
186,10월 코스닥 개인 거래비중 80%..약 20년만에 최저,http://m.finance.daum.net/m/news/news_content....,(서울=연합뉴스) 권수현 기자 = 국내 증시가 글로벌 금융위기 이후 최악의 급락을 ...
187,"[단독]새우깡도 오른다..농심, 과자 20개 브랜드 '가격 최대 9.1% 인상'",http://m.finance.daum.net/m/news/news_content....,[아시아경제 이선애 기자] 주요 식품 가격이 줄줄이 오르고 있는 가운데 과자 가격도...
188,대의원에게 소주 1박스·쌀 1포대 전달..조합 임원 '당선무효형',http://m.finance.daum.net/m/news/news_content....,(춘천=연합뉴스) 이재현 기자 = 대의원을 찾아가 소주 1박스와 쌀 1포대를 기부한...


In [8]:
df.to_csv('{}_with_link_daum.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False),index=False)